# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/maxcarey/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/maxcarey/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/maxcarey/Documents/code-aie7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/maxcarey/Documents/code-aie7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/maxcarey/Documents/code-aie7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [10]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [11]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'dc3c5d'. Skipping!
Property 'summary' already exists in node 'bb9bd4'. Skipping!
Property 'summary' already exists in node '9aac1a'. Skipping!
Property 'summary' already exists in node '56f7c5'. Skipping!
Property 'summary' already exists in node 'defc65'. Skipping!
Property 'summary' already exists in node '54525d'. Skipping!
Property 'summary' already exists in node '18e442'. Skipping!
Property 'summary' already exists in node '662798'. Skipping!
Property 'summary' already exists in node 'ff96ad'. Skipping!
Property 'summary' already exists in node '843c31'. Skipping!
Property 'summary' already exists in node '8edfef'. Skipping!
Property 'summary' already exists in node 'b9d36b'. Skipping!
Property 'summary' already exists in node '659e0a'. Skipping!
Property 'summary' already exists in node '88d931'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'dc3c5d'. Skipping!
Property 'summary_embedding' already exists in node 'bb9bd4'. Skipping!
Property 'summary_embedding' already exists in node '54525d'. Skipping!
Property 'summary_embedding' already exists in node '56f7c5'. Skipping!
Property 'summary_embedding' already exists in node '18e442'. Skipping!
Property 'summary_embedding' already exists in node 'defc65'. Skipping!
Property 'summary_embedding' already exists in node '9aac1a'. Skipping!
Property 'summary_embedding' already exists in node '8edfef'. Skipping!
Property 'summary_embedding' already exists in node '843c31'. Skipping!
Property 'summary_embedding' already exists in node '88d931'. Skipping!
Property 'summary_embedding' already exists in node '659e0a'. Skipping!
Property 'summary_embedding' already exists in node '662798'. Skipping!
Property 'summary_embedding' already exists in node 'ff96ad'. Skipping!
Property 'summary_embedding' already exists in node 'b9d36b'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 0, relationships: 0)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

SingleHopSpecificQuerySynthesizer: Generates straightforward, single-step questions that target specific facts or details from the data. For example, "What is the interest rate for Direct Subsidized Loans?"

MultiHopAbstractQuerySynthesizer: Creates questions that require connecting multiple pieces of information but focus on higher-level concepts or patterns. For example, "How does the FAFSA verification process impact loan disbursement timing?"

MultiHopSpecificQuerySynthesizer: Produces questions that need multiple specific pieces of information to be combined for a complete answer. For example, "What are the eligibility requirements for both Direct Subsidized and Unsubsidized loans, and how do they differ?"



Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,U S Department Education help students with fe...,[Application and Verification Guide Introducti...,The U.S. Department of Education (the Departme...,single_hop_specifc_query_synthesizer
1,What recent changes have been made to the FAFS...,[Chapter 1: The Application Process We removed...,The chapter mentions that the section for Retu...,single_hop_specifc_query_synthesizer
2,What does the FPS do when it finds incconsiste...,[The FPS also checks the application for possi...,The FPS checks the application for possible in...,single_hop_specifc_query_synthesizer
3,What is the DRN used for in FAFSA processing?,[Output Documents After processing is complete...,The DRN appears on the FAFSA Submission Summar...,single_hop_specifc_query_synthesizer
4,What does IRS stand for?,[2. The disclosure of their FTI by the IRS to ...,The context does not specify what IRS stands f...,single_hop_specifc_query_synthesizer
5,How does the privacy and security of sensitive...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The context explains that the Department of Ed...,multi_hop_abstract_query_synthesizer
6,how does the fafsa form online submission proc...,[<1-hop>\n\nequal the tax filer(s) plus depend...,The context explains that students completing ...,multi_hop_abstract_query_synthesizer
7,"How does the application process, including ve...",[<1-hop>\n\nApplication and Verification Guide...,The application process involves completing th...,multi_hop_abstract_query_synthesizer
8,How does the Department verify the student's S...,[<1-hop>\n\nBecause the Department matches the...,The Department verifies the student's SSN by m...,multi_hop_specific_query_synthesizer
9,How does the Department's authority over the u...,[<1-hop>\n\n2. The disclosure of their FTI by ...,The Department has exclusive authority to obta...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'adfd41'. Skipping!
Property 'summary' already exists in node 'a4b23a'. Skipping!
Property 'summary' already exists in node '416fa5'. Skipping!
Property 'summary' already exists in node '46e1e2'. Skipping!
Property 'summary' already exists in node '4b5b60'. Skipping!
Property 'summary' already exists in node '09a73b'. Skipping!
Property 'summary' already exists in node '316fcb'. Skipping!
Property 'summary' already exists in node '50eb02'. Skipping!
Property 'summary' already exists in node '43a20c'. Skipping!
Property 'summary' already exists in node 'f7a41d'. Skipping!
Property 'summary' already exists in node '0c419d'. Skipping!
Property 'summary' already exists in node '6bb954'. Skipping!
Property 'summary' already exists in node '3d56c8'. Skipping!
Property 'summary' already exists in node 'e3a95a'. Skipping!


...

Property 'summary_embedding' already exists in node 'adfd41'. Skipping!
Property 'summary_embedding' already exists in node 'a4b23a'. Skipping!
Property 'summary_embedding' already exists in node '43a20c'. Skipping!
Property 'summary_embedding' already exists in node '416fa5'. Skipping!
Property 'summary_embedding' already exists in node '0c419d'. Skipping!
Property 'summary_embedding' already exists in node '316fcb'. Skipping!
Property 'summary_embedding' already exists in node '4b5b60'. Skipping!
Property 'summary_embedding' already exists in node 'f7a41d'. Skipping!
Property 'summary_embedding' already exists in node '46e1e2'. Skipping!
Property 'summary_embedding' already exists in node '09a73b'. Skipping!
Property 'summary_embedding' already exists in node '6bb954'. Skipping!
Property 'summary_embedding' already exists in node '50eb02'. Skipping!
Property 'summary_embedding' already exists in node '3d56c8'. Skipping!
Property 'summary_embedding' already exists in node 'e3a95a'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of the School Participation D...,"[Chapter 1 Academic Years, Academic Calendars,...",The School Participation Division oversees com...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(b) about in the regulation?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) relates to weeks of instructio...,single_hop_specifc_query_synthesizer
2,Chapter 3 include clinical work or not?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
3,Federal Work-Study is it a payment period prog...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,Wht are the payment periods and how do the wee...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic year must include a minimum numbe...,multi_hop_abstract_query_synthesizer
5,title iv academic year for 34 cfr 668 3 a what...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic year for Title IV purposes is def...,multi_hop_abstract_query_synthesizer
6,How do the regulatory citations 34 CFR 668.3(a...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The regulatory citation 34 CFR 668.3(a) specif...,multi_hop_abstract_query_synthesizer
7,how instruction time and program hours matter ...,[<1-hop>\n\nboth the credit or clock hours and...,the first segment explains that disbursement o...,multi_hop_abstract_query_synthesizer
8,"In Appendix A and B, how disbursement timing a...",[<1-hop>\n\nboth the credit or clock hours and...,"According to the context, Appendix A explains ...",multi_hop_specific_query_synthesizer
9,How do Volume 7 and Volume 8 relate to the inc...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 7 provides guidance on calculating Pell...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [32]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [17]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [20]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [22]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [25]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Based on the provided context, the kinds of loans available include:\n\n- Direct Subsidized Loans (available only to undergraduate students)\n- Direct Unsubsidized Loans\n- Direct PLUS Loans (also referred to as student Federal PLUS Loans or parent Direct PLUS Loans)\n- Subsidized and Unsubsidized Federal Stafford Loans (made under the Federal Family Education Loan (FFEL) Program before July 1, 2010)\n- Federal SLS Loans\n- Federal PLUS Loans (also under FFEL Program before July 1, 2010)\n\nAdditionally, consolidation loans such as Direct Consolidation Loans or Federal Consolidation Loans exist.\n\nTherefore, the main loans currently available are Direct Subsidized Loans, Direct Unsubsidized Loans, and Direct PLUS Loans.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [26]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [31]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

## LangSmith Evaluation

In [33]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'cooked-curve-65' at:
https://smith.langchain.com/o/add79bbe-c927-4cfa-8dbd-3fe101dd7548/datasets/af015d53-666f-42f6-a505-e9401eb372fc/compare?selectedSessions=c89e7334-9201-4912-bf38-5fcb8f282b08




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the definitions of academic years and i...,"Based on the provided context, the definitions...",None,The definitions provided in Volume 2 and Volum...,1,1,0,9.662457,b19ab855-41aa-4b56-9b38-e38b5be3a9fd,6226b841-2f26-46dd-9eab-1718b8ae1024
1,How does Volume 8 relate to the inclusion of c...,"Based on the provided context, Volume 8, Chapt...",None,Volume 8 provides additional guidance on certa...,0,0,0,9.656916,175366d4-914b-46d5-9ce9-118ed788ce12,14f5253c-23d8-4cc2-b728-8512e50c6487
2,How do Volume 7 and Volume 8 relate to the inc...,Based on the provided context:\n\nVolume 7 (Th...,None,Volume 7 provides guidance on calculating Pell...,1,1,0,7.843390,1f7653a5-c4b6-4fbf-93b7-fe5103dc2952,e69f79bf-0295-4e91-90f2-80d424d98a21
3,"In Appendix A and B, how disbursement timing a...",Based on the provided context:\n\n- Appendix A...,None,"According to the context, Appendix A explains ...",1,0,0,7.186034,9e86cbc4-efac-4792-8835-b0a316d3d7c2,e06594af-8620-4f19-9ec6-b8138b1e2bac
4,how instruction time and program hours matter ...,"Based on the provided context, instructional t...",None,the first segment explains that disbursement o...,1,1,0,7.465548,d09df6c2-01a0-48dc-a058-8decf7a23474,56abef0d-7aa9-4204-a084-cf3e9de4c9bb
5,How do the regulatory citations 34 CFR 668.3(a...,The regulatory citations 34 CFR 668.3(a) and 3...,None,The regulatory citation 34 CFR 668.3(a) specif...,1,1,0,5.824912,f6820696-177b-4225-8a36-bb03650c038d,16f0709d-521e-4bd2-abdd-d52fa22dd089
6,title iv academic year for 34 cfr 668 3 a what...,The Title IV academic year for 34 CFR 668.3(a)...,None,The academic year for Title IV purposes is def...,1,0,0,2.381802,2c45c758-a50e-40ee-9ccc-d905bb8aceef,2cfc5cbd-8e91-446b-a066-1b13e49ab7fd
7,Wht are the payment periods and how do the wee...,Payment periods are defined by both credit or ...,None,The academic year must include a minimum numbe...,1,1,0,6.678244,3702cda5-86d8-4ea1-a9b4-c33b4ce1ebf1,262c6845-70ce-4733-91b1-5d9c58eb0c4c
8,Federal Work-Study is it a payment period prog...,"Based on the provided context, Federal Work-St...",None,The payment period is applicable to all Title ...,1,1,0,2.492587,9a92c89d-9311-4ee6-bef7-98d6f2d48098,8e553ba8-e095-448c-bfa2-e50bd101b6f2
9,Chapter 3 include clinical work or not?,"Yes, Chapter 3 includes guidance on the inclus...",None,Inclusion of Clinical Work in a Standard Term ...,1,0,0,2.057844,a9dcc602-1029-4587-b7f3-a09640941257,e41faaa8-ffba-4db9-8a57-0c8c22c92902


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [34]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [35]:
rag_documents = docs

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [37]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [38]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [39]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [40]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [41]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's really important to understand the types of loans that can help support your educational journey. Based on the information provided, there are several kinds of loans available:\n\n1. **Direct Subsidized Loans**: These are loans based on your financial need, and the government may pay the interest while you're in school. You can receive up to the maximum subsidized loan amount depending on your year and dependency status.\n\n2. **Direct Unsubsidized Loans**: These loans are not based on financial need, and interest accrues while you're in school. You may qualify for these loans regardless of your financial situation, especially if you are independent or if a dependent student's parent cannot obtain a Direct PLUS Loan.\n\n3. **Direct PLUS Loans**: Parents of dependent students can apply for these loans to cover the cost of attendance, as long as they meet eligibility requirements. These loans have no fixed limits but can't exceed the student's cost of at

Finally, we can evaluate the new chain on the same test set!

In [42]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'earnest-law-70' at:
https://smith.langchain.com/o/add79bbe-c927-4cfa-8dbd-3fe101dd7548/datasets/af015d53-666f-42f6-a505-e9401eb372fc/compare?selectedSessions=74a97d6b-76b2-4ab4-9621-7fd040aaa711




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do the definitions of academic years and i...,Thank you for your thoughtful question. I unde...,None,The definitions provided in Volume 2 and Volum...,1,1,1,7.906012,b19ab855-41aa-4b56-9b38-e38b5be3a9fd,9d93f14f-68c7-4666-83c4-8a46319fe23a
1,How does Volume 8 relate to the inclusion of c...,Thank you for your thoughtful question. Based ...,None,Volume 8 provides additional guidance on certa...,1,1,1,8.721301,175366d4-914b-46d5-9ce9-118ed788ce12,a7bddfdb-40c3-45b9-8459-7b7be37cbef9
2,How do Volume 7 and Volume 8 relate to the inc...,Thank you for your thoughtful question. Based ...,None,Volume 7 provides guidance on calculating Pell...,1,1,1,7.486489,1f7653a5-c4b6-4fbf-93b7-fe5103dc2952,d46a71a0-4c06-47d3-8bdf-cbeb54bbaa28
3,"In Appendix A and B, how disbursement timing a...",Thank you for your thoughtful question about h...,None,"According to the context, Appendix A explains ...",1,0,1,11.084031,9e86cbc4-efac-4792-8835-b0a316d3d7c2,9569a8f3-bd48-495d-8a4f-3992242e046c
4,how instruction time and program hours matter ...,Thank you for your thoughtful question. Based ...,None,the first segment explains that disbursement o...,1,0,1,8.666036,d09df6c2-01a0-48dc-a058-8decf7a23474,18425f73-fe64-4019-95f2-8dc1a6e16dff
5,How do the regulatory citations 34 CFR 668.3(a...,Thank you for your thoughtful question. It's c...,None,The regulatory citation 34 CFR 668.3(a) specif...,1,0,1,6.091505,f6820696-177b-4225-8a36-bb03650c038d,d7c51bdf-79f5-419c-bdc3-e22fb82aa32c
6,title iv academic year for 34 cfr 668 3 a what...,Thank you for asking such an important questio...,None,The academic year for Title IV purposes is def...,1,0,1,7.439683,2c45c758-a50e-40ee-9ccc-d905bb8aceef,203a46ac-6679-421d-bb2c-03fd2fa91ea7
7,Wht are the payment periods and how do the wee...,Thank you for your thoughtful question—navigat...,None,The academic year must include a minimum numbe...,1,1,1,9.484757,3702cda5-86d8-4ea1-a9b4-c33b4ce1ebf1,13c3900a-80c1-4393-906f-85e5b9fe37dc
8,Federal Work-Study is it a payment period prog...,Thank you for your question—it's important to ...,None,The payment period is applicable to all Title ...,1,1,1,3.030106,9a92c89d-9311-4ee6-bef7-98d6f2d48098,2d1d96b5-46b8-4118-a309-5c2624ebbaa5
9,Chapter 3 include clinical work or not?,Thank you for your question. Based on the cont...,None,Inclusion of Clinical Work in a Standard Term ...,1,1,1,3.421321,a9dcc602-1029-4587-b7f3-a09640941257,4bf4994d-7036-4085-a1e1-db039a14bed6


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.